## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)

# Definitions of Satellite Galaxies and Isolated Galaxies

## Motivation

To illustrate the two definitions, I plot the subhalos, projected onto a plane, and color satellites and isolated galaxies with different colors. 

---

## Set Parameters for the Plots

Choose the snapshot and the simulation, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
sim_id = "V1_MR_curvaton_p082_fix"
snap_id = 127

m31 = (1,0)
mw = (1,1)

Set the maximum distances for satellite galaxies and isolated galaxies (for the definitions by distance):

In [ ]:
maxds = 300
maxdi = 2000 # Maximum distance from LG centre for isolated

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
# Get data:
snap = snapshot.Snapshot(sim_id, snap_id)
cops = snap.get_subhalos("CentreOfPotential") * units.cm.to(units.Mpc)

Compute the masking arrays to select satellites or isolated galaxies from subhalos, and save these to the dictionary "selections":

In [ ]:
selections = {}

# To prune potential spurious:
mask_prune_vmax = dataset_comp.prune_vmax(snap)

# Split into satellites and isolated:
masks_sat_d, mask_isol_d = dataset_comp.split_satellites_by_distance(
    snap, m31, mw, max_dist_sat=maxds, max_dist_isol=maxdi)
masks_sat_gn, mask_isol_gn = dataset_comp.split_satellites_by_group_number(
    snap, m31, mw)

selections['By_distance'] = {'M31_satellites': masks_sat_d[0][mask_prune_vmax],
                             'MW_satellites': masks_sat_d[1][mask_prune_vmax],
                             'Isolated': mask_isol_d[mask_prune_vmax]}
selections['By_GN'] = {'M31_satellites': masks_sat_gn[0][mask_prune_vmax],
                       'MW_satellites': masks_sat_gn[1][mask_prune_vmax],
                       'Isolated': mask_isol_gn[mask_prune_vmax]}

Compute plane projections of subhalo positions:

In [ ]:
cop_m31 = cops[snap.index_of_halo(m31[0], m31[1])]
cop_mw = cops[snap.index_of_halo(mw[0], mw[1])]
LG_centre = dataset_comp.compute_LG_centre(snap, m31, mw) \
                * units.cm.to(units.Mpc)

# Compute the unit separation vector:
r = cop_m31 - cop_mw
print(np.linalg.norm(r))
r = r / np.linalg.norm(r)

# Compute a unit vector orthogonal to r, n:
a = 1; b = 1
n = np.array([-1/r[0] * (a*r[1] + b*r[2]), a, b])
n = n / np.linalg.norm(n)

# Project onto the plane orthogonal to n:
nstack = np.vstack([n] * np.size(cops, axis=0)).T
n_comp = np.dot(cops, n) # Component in the direction of n 
cops_proj = cops - (n_comp * nstack).T

copp_m31 = cops_proj[snap.index_of_halo(m31[0], m31[1])]
copp_mw = cops_proj[snap.index_of_halo(mw[0], mw[1])]
rp = copp_m31 - copp_mw
print(np.linalg.norm(rp))

In [ ]:
# OLD

sim_id = "V1_MR_curvaton_p082_fix"
m31 = (1, 0)
mw = (1, 1)
snap = snapshot_obj.Snapshot(sim_id, 127)
cops = snap.get_subhalos("CentreOfPotential") * units.cm.to(units.Mpc)
mask_vmax = dataset_compute.prune_vmax(snap)

masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(snap, m31, mw)
split_cops_by_r = {"M31_satellites": cops[np.logical_and(masks_sat[0], mask_vmax)],
                   "MW_satellites": cops[np.logical_and(masks_sat[1], mask_vmax)],
                   "isolated": cops[np.logical_and(mask_isol, mask_vmax)]}

masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(snap, m31, mw)
split_cops_by_gn = {"M31_satellites": cops[np.logical_and(masks_sat[0], mask_vmax)],
                    "MW_satellites": cops[np.logical_and(masks_sat[1], mask_vmax)],
                    "isolated": cops[np.logical_and(mask_isol, mask_vmax)]}

---

## Plot 

Plot subhalos with vmax < lowm in the upper row, and the more massive ones in the lower one. Plot two columns for both simulations.

First, set figure parameters:

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Side length (in Mpc) of each subfigure:
ax_size = 5

s = 0.1 # marker size
lw = 0.3 # line width

In [ ]:
def circle(centre_x, centre_y, r, n):
    t = 4*np.pi/n * np.arange(n/2)
    x = centre_x + r * np.cos(t)
    y = centre_y + r * np.sin(t)
    return x, y

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(nrows=2, ncols=2, sharey='row', sharex='col', 
                         subplot_kw=dict(box_aspect=1), figsize=(5,5))
plt.subplots_adjust(wspace=0.04)
plt.subplots_adjust(hspace=0.04)

# Set axis labels:
axes[1,0].set_xlabel("$x [\mathrm{Mpc}]$")
axes[1,1].set_xlabel("$x [\mathrm{Mpc}]$")
axes[0,0].set_ylabel("$y [\mathrm{Mpc}]$")
axes[1,0].set_ylabel("$z [\mathrm{Mpc}]$")

# Set axis limits:
axes[1,0].set_xlim(LG_centre[0] - ax_size/2, 
                   LG_centre[0] + ax_size/2)
axes[1,1].set_xlim(LG_centre[0] - ax_size/2, 
                   LG_centre[0] + ax_size/2)
axes[0,0].set_ylim(LG_centre[1] - ax_size/2, 
                   LG_centre[1] + ax_size/2)
axes[1,0].set_ylim(LG_centre[2] - ax_size/2, 
                   LG_centre[2] + ax_size/2)

axes[0,0].set_title("By group number")
axes[0,1].set_title("By distance")


# Select projection plane:
x = 0; y = 1; z=2


# Plot colored by group number:
axes[0,0].scatter(cops[selections['By_GN']['Isolated']][:,x],
                  cops[selections['By_GN']['Isolated']][:,y],
                  c='gray', s=s)
axes[0,0].scatter(cops[selections['By_GN']['M31_satellites']][:,x],
                  cops[selections['By_GN']['M31_satellites']][:,y],
                  c='pink', s=s)
axes[0,0].scatter(cops[selections['By_GN']['MW_satellites']][:,x],
                  cops[selections['By_GN']['MW_satellites']][:,y],
                  c='lightblue', s=s)
axes[1,0].scatter(cops[selections['By_GN']['Isolated']][:,x],
                  cops[selections['By_GN']['Isolated']][:,z],
                  c='gray', s=s)
axes[1,0].scatter(cops[selections['By_GN']['M31_satellites']][:,x],
                  cops[selections['By_GN']['M31_satellites']][:,z],
                  c='pink', s=s)
axes[1,0].scatter(cops[selections['By_GN']['MW_satellites']][:,x],
                  cops[selections['By_GN']['MW_satellites']][:,z],
                  c='lightblue', s=s)

# Plot colored by distance:
axes[0,1].scatter(cops[selections['By_distance']['Isolated']][:,x],
                  cops[selections['By_distance']['Isolated']][:,y],
                  c='gray', s=s)
axes[0,1].scatter(cops[selections['By_distance']['M31_satellites']][:,x],
                  cops[selections['By_distance']['M31_satellites']][:,y],
                  c='pink', s=s)
axes[0,1].scatter(cops[selections['By_distance']['MW_satellites']][:,x],
                  cops[selections['By_distance']['MW_satellites']][:,y],
                  c='lightblue', s=s)
axes[1,1].scatter(cops[selections['By_distance']['Isolated']][:,x],
                  cops[selections['By_distance']['Isolated']][:,z],
                  c='gray', s=s)
axes[1,1].scatter(cops[selections['By_distance']['M31_satellites']][:,x],
                  cops[selections['By_distance']['M31_satellites']][:,z],
                  c='pink', s=s)
axes[1,1].scatter(cops[selections['By_distance']['MW_satellites']][:,x],
                  cops[selections['By_distance']['MW_satellites']][:,z],
                  c='lightblue', s=s)

In [ ]:
# Add annotations:
for x, y, axx, axy in zip([0,0,0,0], [1,1,2,2], [0,0,1,1], [0,1,0,1]):
    # Plot M31 and circle:
    axes[axx, axy].scatter(cop_m31[x], cop_m31[y], c='red', s=s)
    x_circ, y_circ = circle(cop_m31[x], cop_m31[y], 0.3, 10000)    
    axes[axx, axy].plot(x_circ, y_circ, c='red', linestyle='dashed', linewidth=lw)
    
    # Plot MW and circle:
    axes[axx, axy].scatter(cop_mw[x], cop_mw[y], c='blue', s=s)
    x_circ, y_circ = circle(cop_mw[x], cop_mw[y], 0.3, 10000)
    axes[axx, axy].plot(x_circ, y_circ, c='blue', linestyle='dashed', linewidth=lw)
    
    # Plot 2 Mpc circle around LG centre:
    x_circ, y_circ = circle(LG_centre[x], LG_centre[y], 2, 10000)
    axes[axx, axy].plot(x_circ, y_circ, c='k', linestyle='dashed', linewidth=lw)
    
fig

### Save the Figure

In [ ]:
# Construct saving location:
filename = 'Distinction_comparison_{}.png'.format(sim_id)    
home = os.path.abspath(os.path.join('..'))
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)
    
fig.savefig(filename, dpi=300)